In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

print("="*80)
print("FINANCIAL FEASIBILITY ANALYSIS FOR TOP 20 WIND FARM SITES")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
OUTPUT_DIR = BASE_DIR / 'outputs'

INPUT_CSV = OUTPUT_DIR / 'optimal_sites_top20_GB_5criteria_detailed.csv'
OUTPUT_FINANCIAL_CSV = OUTPUT_DIR / 'wind_farm_financial_analysis.csv'
OUTPUT_SUMMARY_CSV = OUTPUT_DIR / 'wind_farm_financial_summary.csv'

P = 500
REVENUE_RATE = 65
T = 25
CAPEX_BASE = 1.24
CAPEX_GRID_RATE = 2200
OPEX_FIXED = 26000
OPEX_VAR = 6.0
CF_REF = 0.45
CF_MAX = 0.55
DISCOUNT_RATE = 0.07
BETA = 2
ALPHA_ROAD = 0.02
ALPHA_SLOPE = 0.03
S0 = 1.0

print(f"\nFinancial Assumptions:")
print(f"  Project Capacity (P): {P} MW")
print(f"  Revenue Rate: £{REVENUE_RATE}/MWh")
print(f"  Project Lifetime (T): {T} years")
print(f"  Base CapEx: £{CAPEX_BASE} M/MW")
print(f"  Grid Connection: £{CAPEX_GRID_RATE}/MW·km")
print(f"  Fixed OPEX: £{OPEX_FIXED}/MW/year")
print(f"  Variable OPEX: £{OPEX_VAR}/MWh")
print(f"  Reference CF: {CF_REF} ({CF_REF*100}%)")
print(f"  Max CF: {CF_MAX} ({CF_MAX*100}%)")
print(f"  Discount Rate: {DISCOUNT_RATE} ({DISCOUNT_RATE*100}%)")
print(f"  Road Cost Adder (α_road): {ALPHA_ROAD} ({ALPHA_ROAD*100}%)")
print(f"  Slope Cost Adder (α_slope): {ALPHA_SLOPE} ({ALPHA_SLOPE*100}%)")

print("\n" + "="*80)
print("STEP 1: LOAD SITE DATA")
print("="*80)

df = pd.read_csv(INPUT_CSV)
print(f"Loaded {len(df)} sites from {INPUT_CSV.name}")

U_remaining = 6.30
print(f"\nReference wind speed (U_remaining): {U_remaining:.2f} m/s")

print("\n" + "="*80)
print("STEP 2: CALCULATE CAPACITY FACTORS")
print("="*80)

df['RI'] = df['wind_speed_ms'] / U_remaining
df['CF'] = np.minimum(CF_REF * (df['RI'] ** BETA), CF_MAX)

print(f"CF Range: {df['CF'].min():.4f} - {df['CF'].max():.4f}")
print(f"Mean CF: {df['CF'].mean():.4f}")

print("\n" + "="*80)
print("STEP 3: CALCULATE ANNUAL ENERGY PRODUCTION")
print("="*80)

df['AEP_MWh_per_year'] = P * 8760 * df['CF']

print(f"AEP Range: {df['AEP_MWh_per_year'].min():,.0f} - {df['AEP_MWh_per_year'].max():,.0f} MWh/year")
print(f"Mean AEP: {df['AEP_MWh_per_year'].mean():,.0f} MWh/year")

print("\n" + "="*80)
print("STEP 4: CALCULATE ANNUAL REVENUE")
print("="*80)

df['Revenue_M_per_year'] = df['AEP_MWh_per_year'] * REVENUE_RATE / 1e6

print(f"Revenue Range: £{df['Revenue_M_per_year'].min():.2f}M - £{df['Revenue_M_per_year'].max():.2f}M per year")
print(f"Mean Revenue: £{df['Revenue_M_per_year'].mean():.2f}M per year")

print("\n" + "="*80)
print("STEP 5: CALCULATE ANNUAL OPEX")
print("="*80)

df['OPEX_M_per_year'] = (OPEX_FIXED * P + OPEX_VAR * df['AEP_MWh_per_year']) / 1e6

print(f"OPEX Range: £{df['OPEX_M_per_year'].min():.2f}M - £{df['OPEX_M_per_year'].max():.2f}M per year")
print(f"Mean OPEX: £{df['OPEX_M_per_year'].mean():.2f}M per year")

print("\n" + "="*80)
print("STEP 6: CALCULATE CAPEX")
print("="*80)

CAPEX_BoP_ref = P * CAPEX_BASE

df['m_road'] = 1 + ALPHA_ROAD * df['roads_distance_km']
df['m_slope'] = 1 + ALPHA_SLOPE * np.maximum(0, df['slope_degrees'] - S0)

df['CapEx_BoP_M'] = CAPEX_BoP_ref * df['m_road'] * df['m_slope']
df['CapEx_Grid_M'] = CAPEX_GRID_RATE * P * df['grid_distance_km'] / 1e6
df['CapEx_Total_M'] = df['CapEx_BoP_M'] + df['CapEx_Grid_M']

print(f"CapEx BoP Range: £{df['CapEx_BoP_M'].min():.2f}M - £{df['CapEx_BoP_M'].max():.2f}M")
print(f"CapEx Grid Range: £{df['CapEx_Grid_M'].min():.2f}M - £{df['CapEx_Grid_M'].max():.2f}M")
print(f"CapEx Total Range: £{df['CapEx_Total_M'].min():.2f}M - £{df['CapEx_Total_M'].max():.2f}M")
print(f"Mean Total CapEx: £{df['CapEx_Total_M'].mean():.2f}M")

print("\n" + "="*80)
print("STEP 7: CALCULATE NPV")
print("="*80)

df['Annual_Net_CF_M'] = df['Revenue_M_per_year'] - df['OPEX_M_per_year']

discount_factors = [(1 / (1 + DISCOUNT_RATE)**t) for t in range(1, T+1)]
pv_multiplier = sum(discount_factors)

df['NPV_M'] = -df['CapEx_Total_M'] + df['Annual_Net_CF_M'] * pv_multiplier

print(f"NPV Range: £{df['NPV_M'].min():.2f}M - £{df['NPV_M'].max():.2f}M")
print(f"Mean NPV: £{df['NPV_M'].mean():.2f}M")
print(f"Sites with Positive NPV: {(df['NPV_M'] > 0).sum()}/{len(df)}")

print("\n" + "="*80)
print("STEP 8: CALCULATE LCOE")
print("="*80)

discounted_opex = df['OPEX_M_per_year'] * pv_multiplier
discounted_aep = df['AEP_MWh_per_year'] * pv_multiplier

df['LCOE_GBP_per_MWh'] = (df['CapEx_Total_M'] + discounted_opex) / discounted_aep * 1e6

print(f"LCOE Range: £{df['LCOE_GBP_per_MWh'].min():.2f}/MWh - £{df['LCOE_GBP_per_MWh'].max():.2f}/MWh")
print(f"Mean LCOE: £{df['LCOE_GBP_per_MWh'].mean():.2f}/MWh")
print(f"Sites with LCOE < Revenue Rate (£{REVENUE_RATE}/MWh): {(df['LCOE_GBP_per_MWh'] < REVENUE_RATE).sum()}/{len(df)}")

print("\n" + "="*80)
print("STEP 9: CALCULATE PAYBACK PERIOD")
print("="*80)

df['Simple_Payback_Years'] = df['CapEx_Total_M'] / df['Annual_Net_CF_M']

print(f"Payback Period Range: {df['Simple_Payback_Years'].min():.2f} - {df['Simple_Payback_Years'].max():.2f} years")
print(f"Mean Payback Period: {df['Simple_Payback_Years'].mean():.2f} years")

print("\n" + "="*80)
print("STEP 10: PREPARE OUTPUT DATAFRAMES")
print("="*80)

output_df = df[[
    'rank', 'x', 'y', 'wlc_score',
    'wind_speed_ms', 'slope_degrees', 'grid_distance_km', 'roads_distance_km', 'saturation_score',
    'CF', 'AEP_MWh_per_year', 
    'Revenue_M_per_year', 'OPEX_M_per_year', 'Annual_Net_CF_M',
    'CapEx_BoP_M', 'CapEx_Grid_M', 'CapEx_Total_M',
    'NPV_M', 'LCOE_GBP_per_MWh', 'Simple_Payback_Years'
]].copy()

output_df['rank'] = output_df['rank'].astype(int)

output_df = output_df.round({
    'wlc_score': 2, 'wind_speed_ms': 2, 'slope_degrees': 2,
    'grid_distance_km': 2, 'roads_distance_km': 2, 'saturation_score': 2,
    'CF': 4, 'AEP_MWh_per_year': 0,
    'Revenue_M_per_year': 2, 'OPEX_M_per_year': 2, 'Annual_Net_CF_M': 2,
    'CapEx_BoP_M': 2, 'CapEx_Grid_M': 2, 'CapEx_Total_M': 2,
    'NPV_M': 2, 'LCOE_GBP_per_MWh': 2, 'Simple_Payback_Years': 2
})

output_df.to_csv(OUTPUT_FINANCIAL_CSV, index=False)
print(f"Saved detailed results: {OUTPUT_FINANCIAL_CSV.name}")

summary_stats = {
    'Metric': [
        'Capacity Factor',
        'Annual Energy Production (MWh/yr)',
        'Annual Revenue (£M/yr)',
        'Annual OPEX (£M/yr)',
        'Annual Net Cash Flow (£M/yr)',
        'Total CapEx (£M)',
        'NPV (£M)',
        'LCOE (£/MWh)',
        'Simple Payback (years)'
    ],
    'Min': [
        f"{df['CF'].min():.4f}",
        f"{df['AEP_MWh_per_year'].min():,.0f}",
        f"{df['Revenue_M_per_year'].min():.2f}",
        f"{df['OPEX_M_per_year'].min():.2f}",
        f"{df['Annual_Net_CF_M'].min():.2f}",
        f"{df['CapEx_Total_M'].min():.2f}",
        f"{df['NPV_M'].min():.2f}",
        f"{df['LCOE_GBP_per_MWh'].min():.2f}",
        f"{df['Simple_Payback_Years'].min():.2f}"
    ],
    'Max': [
        f"{df['CF'].max():.4f}",
        f"{df['AEP_MWh_per_year'].max():,.0f}",
        f"{df['Revenue_M_per_year'].max():.2f}",
        f"{df['OPEX_M_per_year'].max():.2f}",
        f"{df['Annual_Net_CF_M'].max():.2f}",
        f"{df['CapEx_Total_M'].max():.2f}",
        f"{df['NPV_M'].max():.2f}",
        f"{df['LCOE_GBP_per_MWh'].max():.2f}",
        f"{df['Simple_Payback_Years'].max():.2f}"
    ],
    'Mean': [
        f"{df['CF'].mean():.4f}",
        f"{df['AEP_MWh_per_year'].mean():,.0f}",
        f"{df['Revenue_M_per_year'].mean():.2f}",
        f"{df['OPEX_M_per_year'].mean():.2f}",
        f"{df['Annual_Net_CF_M'].mean():.2f}",
        f"{df['CapEx_Total_M'].mean():.2f}",
        f"{df['NPV_M'].mean():.2f}",
        f"{df['LCOE_GBP_per_MWh'].mean():.2f}",
        f"{df['Simple_Payback_Years'].mean():.2f}"
    ],
    'Median': [
        f"{df['CF'].median():.4f}",
        f"{df['AEP_MWh_per_year'].median():,.0f}",
        f"{df['Revenue_M_per_year'].median():.2f}",
        f"{df['OPEX_M_per_year'].median():.2f}",
        f"{df['Annual_Net_CF_M'].median():.2f}",
        f"{df['CapEx_Total_M'].median():.2f}",
        f"{df['NPV_M'].median():.2f}",
        f"{df['LCOE_GBP_per_MWh'].median():.2f}",
        f"{df['Simple_Payback_Years'].median():.2f}"
    ]
}

summary_df = pd.DataFrame(summary_stats)
summary_df.to_csv(OUTPUT_SUMMARY_CSV, index=False)
print(f"Saved summary statistics: {OUTPUT_SUMMARY_CSV.name}")

print("\n" + "="*80)
print("STEP 11: DISPLAY KEY RESULTS")
print("="*80)

print(f"\nTop 5 Sites by NPV:")
print("="*80)
top5_npv = output_df.nlargest(5, 'NPV_M')[['rank', 'wlc_score', 'wind_speed_ms', 'CF', 'CapEx_Total_M', 'NPV_M', 'LCOE_GBP_per_MWh']]
print(top5_npv.to_string(index=False))

print(f"\nTop 5 Sites by Lowest LCOE:")
print("="*80)
top5_lcoe = output_df.nsmallest(5, 'LCOE_GBP_per_MWh')[['rank', 'wlc_score', 'wind_speed_ms', 'CF', 'CapEx_Total_M', 'NPV_M', 'LCOE_GBP_per_MWh']]
print(top5_lcoe.to_string(index=False))

print("\n" + "="*80)
print("FINANCIAL FEASIBILITY SUMMARY")
print("="*80)

total_capacity = len(df) * P
total_npv = df['NPV_M'].sum()
total_capex = df['CapEx_Total_M'].sum()
total_annual_generation = df['AEP_MWh_per_year'].sum()

print(f"\nPortfolio Overview:")
print(f"  Total Installed Capacity: {total_capacity:,} MW ({total_capacity/1000:.1f} GW)")
print(f"  Number of Sites: {len(df)}")
print(f"  Total CapEx: £{total_capex:,.2f}M (£{total_capex/1000:.2f}B)")
print(f"  Total Portfolio NPV: £{total_npv:,.2f}M (£{total_npv/1000:.2f}B)")
print(f"  Total Annual Generation: {total_annual_generation:,.0f} MWh/year")
print(f"  Average CF: {df['CF'].mean()*100:.2f}%")
print(f"  Average LCOE: £{df['LCOE_GBP_per_MWh'].mean():.2f}/MWh")
print(f"  Average Payback Period: {df['Simple_Payback_Years'].mean():.2f} years")

print(f"\nViability Assessment:")
positive_npv_count = (df['NPV_M'] > 0).sum()
viable_lcoe_count = (df['LCOE_GBP_per_MWh'] < REVENUE_RATE).sum()
print(f"  Sites with Positive NPV: {positive_npv_count}/{len(df)} ({positive_npv_count/len(df)*100:.1f}%)")
print(f"  Sites with LCOE < £{REVENUE_RATE}/MWh: {viable_lcoe_count}/{len(df)} ({viable_lcoe_count/len(df)*100:.1f}%)")

if positive_npv_count == len(df) and viable_lcoe_count == len(df):
    print(f"  ✓ ALL SITES ARE FINANCIALLY VIABLE")
elif positive_npv_count >= len(df) * 0.9:
    print(f"  ✓ MAJORITY OF SITES ARE FINANCIALLY VIABLE")
else:
    print(f"   SOME SITES MAY NOT BE FINANCIALLY VIABLE")

print("\n" + "="*80)
print("COMPLETE DATA TABLE FOR ALL 20 SITES")
print("="*80)

print("\n" + "="*80)
print("TABLE 1: SITE CHARACTERISTICS AND WLC SCORES")
print("="*80)
print("\nRank | WLC Score | Wind (m/s) | Slope (deg) | Grid (km) | Roads (km) | Satur (scr)")
print("-"*88)
for idx, row in output_df.iterrows():
    print(f"{int(row['rank']):4d} | {row['wlc_score']:9.2f} | {row['wind_speed_ms']:10.2f} | "
          f"{row['slope_degrees']:11.2f} | {row['grid_distance_km']:9.2f} | "
          f"{row['roads_distance_km']:10.2f} | {row['saturation_score']:10.2f}")

print("\n" + "="*80)
print("TABLE 2: CAPACITY FACTOR AND ANNUAL ENERGY PRODUCTION")
print("="*80)
print("\nRank | Capacity Factor | AEP (MWh/yr) | Revenue (£M/yr) | OPEX (£M/yr) | Net CF (£M/yr)")
print("-"*95)
for idx, row in output_df.iterrows():
    print(f"{int(row['rank']):4d} | {row['CF']:15.4f} | {row['AEP_MWh_per_year']:12,.0f} | "
          f"{row['Revenue_M_per_year']:15.2f} | {row['OPEX_M_per_year']:12.2f} | "
          f"{row['Annual_Net_CF_M']:14.2f}")

print("\n" + "="*80)
print("TABLE 3: CAPITAL EXPENDITURE BREAKDOWN")
print("="*80)
print("\nRank | CapEx BoP (£M) | CapEx Grid (£M) | CapEx Total (£M) | m_road | m_slope")
print("-"*85)
for idx, row in output_df.iterrows():
    m_road = 1 + ALPHA_ROAD * row['roads_distance_km']
    m_slope = 1 + ALPHA_SLOPE * max(0, row['slope_degrees'] - S0)
    print(f"{int(row['rank']):4d} | {row['CapEx_BoP_M']:14.2f} | {row['CapEx_Grid_M']:15.2f} | "
          f"{row['CapEx_Total_M']:16.2f} | {m_road:6.4f} | {m_slope:7.4f}")

print("\n" + "="*80)
print("TABLE 4: FINANCIAL VIABILITY INDICATORS")
print("="*80)
print("\nRank | NPV (£M) | LCOE (£/MWh) | Payback (years) | Viable?")
print("-"*70)
for idx, row in output_df.iterrows():
    viable = "YES" if row['NPV_M'] > 0 and row['LCOE_GBP_per_MWh'] < REVENUE_RATE else "NO"
    print(f"{int(row['rank']):4d} | {row['NPV_M']:8.2f} | {row['LCOE_GBP_per_MWh']:12.2f} | "
          f"{row['Simple_Payback_Years']:15.2f} | {viable:7s}")

print("\n" + "="*80)
print("TABLE 5: COMPLETE FINANCIAL METRICS (TOP 10 SITES)")
print("="*80)
print("\nRank | Wind | CF     | AEP(GWh) | Rev(£M) | OPEX(£M) | CapEx(£M) | NPV(£M) | LCOE(£/MWh)")
print("-"*105)
for idx, row in output_df.head(10).iterrows():
    print(f"{int(row['rank']):4d} | {row['wind_speed_ms']:4.2f} | {row['CF']:.4f} | "
          f"{row['AEP_MWh_per_year']/1000:8.1f} | {row['Revenue_M_per_year']:7.2f} | "
          f"{row['OPEX_M_per_year']:8.2f} | {row['CapEx_Total_M']:9.2f} | "
          f"{row['NPV_M']:7.2f} | {row['LCOE_GBP_per_MWh']:11.2f}")

print("\n" + "="*80)
print("TABLE 6: COMPLETE FINANCIAL METRICS (RANK 11-20)")
print("="*80)
print("\nRank | Wind | CF     | AEP(GWh) | Rev(£M) | OPEX(£M) | CapEx(£M) | NPV(£M) | LCOE(£/MWh)")
print("-"*105)
for idx, row in output_df.tail(10).iterrows():
    print(f"{int(row['rank']):4d} | {row['wind_speed_ms']:4.2f} | {row['CF']:.4f} | "
          f"{row['AEP_MWh_per_year']/1000:8.1f} | {row['Revenue_M_per_year']:7.2f} | "
          f"{row['OPEX_M_per_year']:8.2f} | {row['CapEx_Total_M']:9.2f} | "
          f"{row['NPV_M']:7.2f} | {row['LCOE_GBP_per_MWh']:11.2f}")

print("\n" + "="*80)
print("TABLE 7: REGIONAL DISTRIBUTION")
print("="*80)

def get_region(y):
    if y >= 800000:
        return "Scotland"
    elif y >= 400000:
        return "Northern England"
    elif y >= 200000:
        return "Central England"
    else:
        return "Southern England/Wales"

output_df['Region'] = output_df['y'].apply(get_region)
regional_summary = output_df.groupby('Region').agg({
    'rank': 'count',
    'CF': 'mean',
    'CapEx_Total_M': 'sum',
    'NPV_M': 'sum',
    'LCOE_GBP_per_MWh': 'mean',
    'AEP_MWh_per_year': 'sum'
}).round(2)

regional_summary.columns = ['Count', 'Avg_CF', 'Total_CapEx_M', 'Total_NPV_M', 'Avg_LCOE', 'Total_AEP_MWh']
print("\nRegion                    | Count | Avg CF | Total CapEx (£M) | Total NPV (£M) | Avg LCOE | Total AEP (GWh/yr)")
print("-"*120)
for region, row in regional_summary.iterrows():
    print(f"{region:24s} | {int(row['Count']):5d} | {row['Avg_CF']:6.4f} | {row['Total_CapEx_M']:16.2f} | "
          f"{row['Total_NPV_M']:14.2f} | {row['Avg_LCOE']:8.2f} | {row['Total_AEP_MWh']/1000:17.1f}")

print("\n" + "="*80)
print("TABLE 8: SENSITIVITY ANALYSIS - TOP 5 vs BOTTOM 5")
print("="*80)

top5 = output_df.head(5)
bottom5 = output_df.tail(5)

print(f"\nMetric                    | Top 5 Sites | Bottom 5 Sites | Difference")
print("-"*75)
print(f"Average WLC Score         | {top5['wlc_score'].mean():11.2f} | {bottom5['wlc_score'].mean():14.2f} | {top5['wlc_score'].mean() - bottom5['wlc_score'].mean():10.2f}")
print(f"Average Wind Speed (m/s)  | {top5['wind_speed_ms'].mean():11.2f} | {bottom5['wind_speed_ms'].mean():14.2f} | {top5['wind_speed_ms'].mean() - bottom5['wind_speed_ms'].mean():10.2f}")
print(f"Average CF                | {top5['CF'].mean():11.4f} | {bottom5['CF'].mean():14.4f} | {top5['CF'].mean() - bottom5['CF'].mean():10.4f}")
print(f"Average CapEx (£M)        | {top5['CapEx_Total_M'].mean():11.2f} | {bottom5['CapEx_Total_M'].mean():14.2f} | {top5['CapEx_Total_M'].mean() - bottom5['CapEx_Total_M'].mean():10.2f}")
print(f"Average NPV (£M)          | {top5['NPV_M'].mean():11.2f} | {bottom5['NPV_M'].mean():14.2f} | {top5['NPV_M'].mean() - bottom5['NPV_M'].mean():10.2f}")
print(f"Average LCOE (£/MWh)      | {top5['LCOE_GBP_per_MWh'].mean():11.2f} | {bottom5['LCOE_GBP_per_MWh'].mean():14.2f} | {top5['LCOE_GBP_per_MWh'].mean() - bottom5['LCOE_GBP_per_MWh'].mean():10.2f}")
print(f"Average Payback (years)   | {top5['Simple_Payback_Years'].mean():11.2f} | {bottom5['Simple_Payback_Years'].mean():14.2f} | {top5['Simple_Payback_Years'].mean() - bottom5['Simple_Payback_Years'].mean():10.2f}")

print("\n" + "="*80)
print("TABLE 9: COMPARISON WITH PAPER EXAMPLES (VALIDATION)")
print("="*80)

validation_sites = [1, 7, 18]
print("\nSite | Paper NPV (£M) | Calculated NPV (£M) | Paper LCOE (£/MWh) | Calculated LCOE (£/MWh)")
print("-"*100)

paper_data = {
    1: {'NPV': 844.29, 'LCOE': 34.93},
    7: {'NPV': 701.44, 'LCOE': 37.67},
    18: {'NPV': 457.97, 'LCOE': 43.16}
}

for site_rank in validation_sites:
    row = output_df[output_df['rank'] == site_rank].iloc[0]
    paper = paper_data[site_rank]
    npv_diff = abs(row['NPV_M'] - paper['NPV'])
    lcoe_diff = abs(row['LCOE_GBP_per_MWh'] - paper['LCOE'])
    print(f"{site_rank:4d} | {paper['NPV']:14.2f} | {row['NPV_M']:19.2f} | "
          f"{paper['LCOE']:18.2f} | {row['LCOE_GBP_per_MWh']:23.2f}")
    if npv_diff < 1.0 and lcoe_diff < 0.1:
        print(f"      ✓ VALIDATED (NPV diff: £{npv_diff:.2f}M, LCOE diff: £{lcoe_diff:.2f}/MWh)")
    else:
        print(f"      ⚠ Check calculation (NPV diff: £{npv_diff:.2f}M, LCOE diff: £{lcoe_diff:.2f}/MWh)")

print("\n" + "="*80)
print("ANALYSIS COMPLETE")
print("="*80)
print(f"\nOutput Files:")
print(f"  1. {OUTPUT_FINANCIAL_CSV.name} - Detailed financial metrics for all 20 sites")
print(f"  2. {OUTPUT_SUMMARY_CSV.name} - Summary statistics")
print(f"\nAll 20 sites have been analyzed with complete financial indicators.")
print(f"Refer to the tables above for comprehensive breakdown of:")
print(f"  • Site characteristics and WLC scores")
print(f"  • Capacity factors and energy production")
print(f"  • Capital expenditure breakdown")
print(f"  • Financial viability indicators")
print(f"  • Regional distribution analysis")
print(f"  • Sensitivity comparisons")
print(f"  • Validation against paper results")
print("="*80)

FINANCIAL FEASIBILITY ANALYSIS FOR TOP 20 WIND FARM SITES

Financial Assumptions:
  Project Capacity (P): 500 MW
  Revenue Rate: £65/MWh
  Project Lifetime (T): 25 years
  Base CapEx: £1.24 M/MW
  Grid Connection: £2200/MW·km
  Fixed OPEX: £26000/MW/year
  Variable OPEX: £6.0/MWh
  Reference CF: 0.45 (45.0%)
  Max CF: 0.55 (55.00000000000001%)
  Discount Rate: 0.07 (7.000000000000001%)
  Road Cost Adder (α_road): 0.02 (2.0%)
  Slope Cost Adder (α_slope): 0.03 (3.0%)

STEP 1: LOAD SITE DATA
Loaded 20 sites from optimal_sites_top20_GB_5criteria_detailed.csv

Reference wind speed (U_remaining): 6.30 m/s

STEP 2: CALCULATE CAPACITY FACTORS
CF Range: 0.4109 - 0.5500
Mean CF: 0.4953

STEP 3: CALCULATE ANNUAL ENERGY PRODUCTION
AEP Range: 1,799,693 - 2,409,000 MWh/year
Mean AEP: 2,169,534 MWh/year

STEP 4: CALCULATE ANNUAL REVENUE
Revenue Range: £116.98M - £156.59M per year
Mean Revenue: £141.02M per year

STEP 5: CALCULATE ANNUAL OPEX
OPEX Range: £23.80M - £27.45M per year
Mean OPEX: £26.02M 